In [2]:
import pandas as pd
import datetime
import backtrader as bt
import numpy as np
import matplotlib.pyplot as plt


In [3]:
def sim_leverage(proxy, leverage=1, expense_ratio = 0.0, initial_value=1.0, start_date=None):
    """
    Simulates a leverage ETF given its proxy, leverage, and expense ratio.
    
    Daily percent change is calculated by taking the daily percent change of
    the proxy, subtracting the daily expense ratio, then multiplying by the leverage.
    """
    val = proxy['Close']
    pct_change = (val - val.shift(1)) / val.shift(1)
    if start_date is not None:
        pct_change = pct_change[pct_change.index > start_date]
    pct_change = (pct_change - expense_ratio / 252) * leverage
    sim = ((1 + pct_change).cumprod() * initial_value).to_frame("Close")
    # sim[0] = initial_value
    for column in ["Open", "High", "Low"]:
        sim[column] = sim["Close"]
    sim["Volume"] = 0
    return sim


In [4]:
def process_yahoo_csv(file_name):
    df = pd.read_csv(file_name, 
                     parse_dates=True,
                     index_col=0)

    price_ratio = df['Adj Close']/df['Close']
    for column in ["Open", "High", "Low", "Close"]:
        df[column] = df[column]*price_ratio

    return df[["Open", "High", "Low", "Close", "Volume"]]


In [5]:
vfinx_df = process_yahoo_csv("VFINX.csv")
vustx_df = process_yahoo_csv("VUSTX.csv")
nasdaq_df = process_yahoo_csv("NASDAQ.csv")
tqqq_df = process_yahoo_csv("TQQQ.csv")
tmf_df = process_yahoo_csv("TMF.csv")
upro_df = process_yahoo_csv("upro.csv")
# nasdaq_df['Close'] /= 100

# upro_sim_df = vfinx_df.copy()
# tmf_sim_df = vustx_df.copy()
# tqqq_sim_df = nasdaq_df.copy()

upro_sim_df = sim_leverage(vfinx_df, leverage=3.0, expense_ratio=0.015) #, initial_value=upro_df.iloc[0]['Close'], start_date=upro_df.index[0])
tmf_sim_df = sim_leverage(vustx_df, leverage=3.0, expense_ratio=0.015) #, initial_value=tmf_df.iloc[0]['Close'],start_date=tmf_df.index[0])
tqqq_sim_df = sim_leverage(nasdaq_df, leverage=3.0, expense_ratio=0.015) #, initial_value=tqqq_df.iloc[0]['Close'],start_date=tqqq_df.index[0])


In [8]:
%matplotlib widget
plt.plot((upro_sim_df['Close']- upro_df['Close'])/upro_df['Close'])
# plt.legend(['UPRO_sim', 'UPRO'])
plt.title('2% expense')
plt.ylabel('sim price error (sim-real)/real')
# plt.yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'sim price error (sim-real)/real')

In [6]:
# resample to month
upro_sim_df = upro_sim_df.groupby(pd.Grouper(freq="M")).last()
tmf_sim_df = tmf_sim_df.groupby(pd.Grouper(freq="M")).last()
tqqq_sim_df = tqqq_sim_df.groupby(pd.Grouper(freq="M")).last()
vfinx_df = vfinx_df.groupby(pd.Grouper(freq="M")).last()
vustx_df = vustx_df.groupby(pd.Grouper(freq="M")).last()
nasdaq_df = nasdaq_df.groupby(pd.Grouper(freq="M")).last()


In [7]:
upro_sim_df.head()

,Close,Open,High,Low,Volume
Date,,,,,
1980-01-31,1.255049,1.255049,1.255049,1.255049,0
1980-02-29,1.258134,1.258134,1.258134,1.258134,0
1980-03-31,0.907847,0.907847,0.907847,0.907847,0
1980-04-30,1.019033,1.019033,1.019033,1.019033,0
1980-05-31,1.187478,1.187478,1.187478,1.187478,0


In [8]:
start = datetime.datetime(1986, 5, 19)
end = datetime.datetime(2020, 4, 20)

upro_sim = bt.feeds.PandasData(dataname=upro_sim_df, fromdate=start, todate=end)
tmf_sim = bt.feeds.PandasData(dataname=tmf_sim_df, fromdate=start, todate=end)
vfinx = bt.feeds.PandasData(dataname=vfinx_df, fromdate=start, todate=end)
tqqq_sim = bt.feeds.PandasData(dataname=tqqq_sim_df, fromdate=start, todate=end)


In [9]:
class BuyAndHold(bt.Strategy):
    def next(self):
        if not self.getposition(self.data).size:
            self.order_target_percent(self.data, target=1.0)


In [10]:
def backtest(datas, strategy, plot=False, **kwargs):
    cerebro = bt.Cerebro()
    for data in datas:
        cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.Returns, timeframe=bt.TimeFrame.Months)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    cerebro.addstrategy(strategy, **kwargs)
    results = cerebro.run()
    if plot:
        cerebro.plot()
    return (results[0].analyzers.drawdown.get_analysis()['max']['drawdown'],
            results[0].analyzers.returns.get_analysis()['rnorm100'],
            results[0].analyzers.sharperatio.get_analysis()['sharperatio'])


In [35]:
dd, cagr, sharpe = backtest([vfinx], BuyAndHold, plot=True)
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


<IPython.core.display.Javascript object>

Max Drawdown: 50.64%
CAGR: 9.15%
Sharpe: 0.605


In [36]:
dd, cagr, sharpe = backtest([upro_sim], BuyAndHold)
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


Max Drawdown: 96.72%
CAGR: 9.94%
Sharpe: 0.460


In [37]:
dd, cagr, sharpe = backtest([tmf_sim], BuyAndHold)
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


Max Drawdown: 49.87%
CAGR: 14.75%
Sharpe: 0.554


In [11]:
class AssetAllocation(bt.Strategy):
    params = (
        ('asset_alloc', None),
    )
    def __init__(self):
        pass
                
    def next(self):
        if self.params.asset_alloc is not None:
            for asset, alloc in zip(self.datas, self.params.asset_alloc):
                self.order_target_percent(asset, target=alloc)


In [17]:
dd, cagr, sharpe = backtest([upro_sim, tmf_sim, tqqq_sim], AssetAllocation, plot=True, asset_alloc=[0.3, 0.5, 0.2])
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


<IPython.core.display.Javascript object>

Max Drawdown: 63.06%
CAGR: 19.78%
Sharpe: 0.672


In [42]:
dd, cagr, sharpe = backtest([upro_sim, tmf_sim, tqqq_sim], AssetAllocation, plot=True,  asset_alloc=[0.3, 0.5, 0.2])
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


<IPython.core.display.Javascript object>

Max Drawdown: 55.47%
CAGR: 17.73%
Sharpe: 0.643


In [43]:
dd, cagr, sharpe = backtest([upro_sim, tmf_sim, tqqq_sim], AssetAllocation, plot=True,  asset_alloc=[0.3, 0.5, 0.2])
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


Max Drawdown: 55.47%
CAGR: 17.73%
Sharpe: 0.643


In [18]:
bt_result = []
for start_year in range(1987, 2020):
    for end_year in range(start_year+2, 2021):

        start = datetime.datetime(start_year, 1, 1)
        end = datetime.datetime(end_year, 1, 1)

        upro_sim = bt.feeds.PandasData(dataname=upro_sim_df, fromdate=start, todate=end)
        tmf_sim = bt.feeds.PandasData(dataname=tmf_sim_df, fromdate=start, todate=end)
        vfinx = bt.feeds.PandasData(dataname=vfinx_df, fromdate=start, todate=end)
        vustx = bt.feeds.PandasData(dataname=vustx_df, fromdate=start, todate=end)
        tqqq_sim = bt.feeds.PandasData(dataname=tqqq_sim_df, fromdate=start, todate=end)
        for pct_equity in range(0, 101, 20):
            ratio_equity = pct_equity/100.0
            # asset_alloc=[ratio_equity*0.6, 1-ratio_equity, ratio_equity*0.4]
            # dd, cagr, sharpe = backtest([upro_sim, tmf_sim, tqqq_sim], AssetAllocation, asset_alloc=asset_alloc)
            asset_alloc=[ratio_equity, 1-ratio_equity]
            dd, cagr, sharpe = backtest([vfinx, vustx], AssetAllocation, asset_alloc=asset_alloc)
            bt_result.append({'start':start, 'end': end, 'cagr': cagr, 
                              'dd':dd, 'sharpe':sharpe, 'pct_equity':pct_equity})
            print(f"Start {start_year}, End {end_year}, %eq %{pct_equity}, Max Drawdown: {dd:.2f}, CAGR: {cagr:.2f}, Sharpe: {sharpe:.3f}")
bt_result = pd.DataFrame(bt_result)
bt_result.to_csv('bench_result_monthly.csv')

Start 1987, End 1989, %eq %0, Max Drawdown: 10.73, CAGR: 2.66, Sharpe: 0.448
Start 1987, End 1989, %eq %20, Max Drawdown: 4.50, CAGR: 5.40, Sharpe: 1.130
Start 1987, End 1989, %eq %40, Max Drawdown: 10.60, CAGR: 4.69, Sharpe: 0.731
Start 1987, End 1989, %eq %60, Max Drawdown: 17.09, CAGR: 2.95, Sharpe: 0.347
Start 1987, End 1989, %eq %80, Max Drawdown: 23.41, CAGR: 2.86, Sharpe: 0.312
Start 1987, End 1989, %eq %100, Max Drawdown: 29.76, CAGR: 0.75, Sharpe: 0.123
Start 1987, End 1990, %eq %0, Max Drawdown: 10.73, CAGR: 7.55, Sharpe: 0.894
Start 1987, End 1990, %eq %20, Max Drawdown: 4.50, CAGR: 10.22, Sharpe: 1.295
Start 1987, End 1990, %eq %40, Max Drawdown: 10.60, CAGR: 10.52, Sharpe: 1.077
Start 1987, End 1990, %eq %60, Max Drawdown: 17.09, CAGR: 10.07, Sharpe: 0.820
Start 1987, End 1990, %eq %80, Max Drawdown: 23.41, CAGR: 10.79, Sharpe: 0.794
Start 1987, End 1990, %eq %100, Max Drawdown: 29.76, CAGR: 10.07, Sharpe: 0.637
Start 1987, End 1991, %eq %0, Max Drawdown: 10.73, CAGR: 7.13

KeyboardInterrupt: 

In [9]:
bt_result = pd.read_csv('bt_result_monthly.csv', parse_dates=True, index_col=0)
bt_result['start'] = pd.to_datetime(bt_result['start'],format='%Y-%m-%d')
bt_result['end'] = pd.to_datetime(bt_result['end'],format='%Y-%m-%d')

In [10]:
bt_result.tail()

,start,end,cagr,dd,sharpe,pct_equity
5803,2018-01-01,2020-01-01,21.669767,25.945466,0.674474,60
5804,2018-01-01,2020-01-01,21.647537,29.724026,0.640853,70
5805,2018-01-01,2020-01-01,21.447458,33.811381,0.612811,80
5806,2018-01-01,2020-01-01,21.495047,37.884303,0.592945,90
5807,2018-01-01,2020-01-01,21.249372,42.747155,0.573758,100


In [11]:
bt_result['horizon'] = (bt_result['end'].dt.year - bt_result['start'].dt.year)
bt_horizon = bt_result[(bt_result['horizon'] % 5 ==0) & (bt_result['horizon'] <= 20)]
horizon5 = bt_result[(bt_result['horizon'] == 5)]

In [24]:
%matplotlib widget
bt_horizon.sort_values('dd').drop_duplicates(['start', 'horizon'],keep='first')['pct_equity'].hist(by=bt_horizon['horizon'], bins=range(-5,106,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x00000265C27896D0>,
      dtype=object)

In [23]:
%matplotlib widget
bt_horizon.sort_values('cagr').drop_duplicates(['start', 'horizon'],keep='last')['pct_equity'].hist(by=bt_horizon['horizon'], bins=range(-5,106,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x00000265C2622A00>,
      dtype=object)

In [14]:
%matplotlib widget
bt_horizon.sort_values('sharpe').drop_duplicates(['start', 'horizon'],keep='last')['pct_equity'].hist(by=bt_horizon['horizon'], bins=range(-5,106,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x00000265BBAD6EB0>,
      dtype=object)

In [32]:
bt_horizon[(bt_horizon['pct_equity']==20)]['dd'].hist(by=bt_horizon['horizon'], bins=range(8,53,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x00000265C6B4D610>,
      dtype=object)

In [30]:
bt_horizon[(bt_horizon['pct_equity']==40)]['cagr'].hist(by=bt_horizon['horizon'], bins=range(-2,43,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x00000265C4EC8370>,
      dtype=object)

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

In [23]:
bt_horizon[(bt_horizon['pct_equity']==100)]['cagr'].hist(by=bt_horizon['horizon'])#, bins=range(-2,43,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001AFEF568C10>,
      dtype=object)

In [24]:
bt_horizon[(bt_horizon['pct_equity']==100)]['dd'].hist(by=bt_horizon['horizon'])#, bins=range(8,53,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001AFED2BBD30>,
      dtype=object)